# ITI105 Machine Learning Project

Team members:
* Lye Suh Jeng (7487427Y)
* Lee Li Neng (6203055B)
* Lim Chan Boon (9704541M)
---
Project Problem: 4 (a) as in suggested project:
* The success of the song can often been measured by whether the song is on the Hit Chart such as Billboard Hot 100. It is important for music labels to know what makes a song successful so that they can focus their budget on making songs that has the highest chance of being successful.
---
We want to solve the problem statement by using the follow steps:
1.   Load dataset
2.   Discover & visualize data to gain insights
3.   Prepare data
4.   Feature scaling
6.   Feature reduction
5.   Split data into train and test datasets
6.   Train, fine tune and evaluate models
7.   Compare performance of models
8.   Deploy the model

# (1) Gather and Load dataset

In [ ]:
# Suppress warnings about too few trees from the early models
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor

df = pd.read_csv('https://raw.githubusercontent.com/dy018/project105/main/song_data.csv')

## 2.1 Split features into numeric and categorical features

In [ ]:
# song_popularity is target. The rest of columns are features
# Get list of features from df
features = df.columns[1:].tolist()

# put features into 2 types: categorical features and numeric features
category_features = ['audio_mode', 'time_signature', 'key']
numeric_features = [feature for feature in features if feature not in category_features]
print(">>> Categorical features are:", category_features)
print(">>> Numeric features are:", numeric_features)


>>> Categorical features are: ['audio_mode', 'time_signature', 'key']
>>> Numeric features are: ['song_popularity', 'song_duration_ms', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'audio_valence']


# (3) Prepare data


In [ ]:
  # Backup original df
  df2 = df.copy()

## (3)(a) Prepare data by removing any duplicat records if any


In [ ]:
# Remove duplicate rows (if any)
print(">>>> Original df's shape:", df.shape)

counter = 0
row,col = df.shape

df2.drop_duplicates(inplace=True)

if df2.shape==(row,col):
    print('>>> The dataset doesn\'t have any duplicates')
else:
    print('>>> Number of duplicates dropped/fixed:', {row-df2.shape[0]})
    print(">>> New df2's shape after removing dupliate rows", df2.shape)

>>>> Original df's shape: (18835, 15)
>>> Number of duplicates dropped/fixed: {3909}
>>> New df2's shape after removing dupliate rows (14926, 15)


## (3)(b) Convert categorical features to numeric using dummy encoding

In [ ]:
# backup df:
df3 = df2.copy()

# Convert categorical features to numeric using dummy encoding
for feature in category_features:
    dummies = pd.get_dummies(df3[feature], prefix=feature)
    df3 = pd.concat([df3, dummies], axis=1)
    df3.drop(feature, axis=1, inplace=True)

print(">>> df's shape after converting categorical features to numeric using dummy encoding:\n", df3.shape)
print(">>> Columns in new df:\n", df3.columns)
print(">>> First 5 records in new df3:\n")
df3.head()

>>> df's shape after converting categorical features to numeric using dummy encoding:
 (14926, 31)
>>> Columns in new df:
 Index(['song_name', 'song_popularity', 'song_duration_ms', 'acousticness',
       'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness',
       'speechiness', 'tempo', 'audio_valence', 'audio_mode_0', 'audio_mode_1',
       'time_signature_0', 'time_signature_1', 'time_signature_3',
       'time_signature_4', 'time_signature_5', 'key_0', 'key_1', 'key_2',
       'key_3', 'key_4', 'key_5', 'key_6', 'key_7', 'key_8', 'key_9', 'key_10',
       'key_11'],
      dtype='object')
>>> First 5 records in new df3:



,song_name,song_popularity,song_duration_ms,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,...,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
0,Boulevard of Broken Dreams,73,262333,0.005520,0.496,0.682,0.000029,0.0589,-4.095,0.0294,...,False,False,False,False,False,False,True,False,False,False
1,In The End,66,216933,0.010300,0.542,0.853,0.000000,0.1080,-6.407,0.0498,...,False,True,False,False,False,False,False,False,False,False
2,Seven Nation Army,76,231733,0.008170,0.737,0.463,0.447000,0.2550,-7.828,0.0792,...,False,False,False,False,False,False,False,False,False,False
3,By The Way,74,216933,0.026400,0.451,0.970,0.003550,0.1020,-4.938,0.1070,...,False,False,False,False,False,False,False,False,False,False
4,How You Remind Me,56,223826,0.000954,0.447,0.766,0.000000,0.1130,-5.065,0.0313,...,False,False,False,False,False,False,False,False,True,False


# 4) Combine features

In [ ]:
  df4 = df3.copy()
  # combine loudness and energy (correlation  = 0.76)
  df4['loudness_energy'] = df4['loudness'] * df4['energy']

  # combine danceability and audio_valence (correlation = 0.33)
  df4['danceability_av'] = df4['danceability'] * df4['audio_valence']

  #drop loudness, energy, danceability, audio_valence
  df4.drop(['song_name','loudness', 'energy', 'danceability', 'audio_valence'], axis=1, inplace=True)

  print(">>> df's shape after combining features:\n", df4.shape)

>>> df's shape after combining features:
 (14926, 28)


In [ ]:
df4.head()

,song_popularity,song_duration_ms,acousticness,instrumentalness,liveness,speechiness,tempo,audio_mode_0,audio_mode_1,time_signature_0,...,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11,loudness_energy,danceability_av
0,73,262333,0.005520,0.000029,0.0589,0.0294,167.060,False,True,False,...,False,False,False,False,True,False,False,False,-2.792790,0.235104
1,66,216933,0.010300,0.000000,0.1080,0.0498,105.256,True,False,False,...,False,False,False,False,False,False,False,False,-5.465171,0.200540
2,76,231733,0.008170,0.447000,0.2550,0.0792,123.881,False,True,False,...,False,False,False,False,False,False,False,False,-3.624364,0.238788
3,74,216933,0.026400,0.003550,0.1020,0.1070,122.444,False,True,False,...,False,False,False,False,False,False,False,False,-4.789860,0.089298
4,56,223826,0.000954,0.000000,0.1130,0.0313,172.011,False,True,False,...,False,False,False,False,False,False,True,False,-3.879790,0.256578


# 5) Feature scaling

In [ ]:
# Apply scaling on columns of df
def scaler(temp_df):
  # Initialize the MinMaxScaler
  min_max_scaler = MinMaxScaler()

  # Fit the scaler to the data and transform it
  min_max_scaled_data = min_max_scaler.fit_transform(temp_df)

  temp_df = pd.DataFrame(min_max_scaled_data, columns=temp_df.columns)
  return temp_df

Y = df4['song_popularity']

X_scaled = scaler(df4.drop(['song_popularity'], axis=1))

print (">>> Display first 5 records of training data after scaling:\n")
X_scaled.head()

>>> Display first 5 records of training data after scaling:



,song_duration_ms,acousticness,instrumentalness,liveness,speechiness,tempo,audio_mode_0,audio_mode_1,time_signature_0,time_signature_1,...,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11,loudness_energy,danceability_av
0,0.140059,0.005541,0.000029,0.049226,0.031243,0.689425,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.714949,0.252207
1,0.114658,0.010340,0.000000,0.099580,0.052922,0.434371,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.539481,0.215128
2,0.122938,0.008202,0.448345,0.250333,0.084166,0.511233,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.660348,0.256159
3,0.114658,0.026505,0.003561,0.093426,0.113709,0.505303,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.583822,0.095794
4,0.118514,0.000957,0.000000,0.104707,0.033262,0.709856,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.643577,0.275243


In [ ]:
print (">>> Display df's statistic after scaling:\n")
X_scaled.describe()

>>> Display df's statistic after scaling:



,song_duration_ms,acousticness,instrumentalness,liveness,speechiness,tempo,audio_mode_0,audio_mode_1,time_signature_0,time_signature_1,...,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11,loudness_energy,danceability_av
count,14926.000000,14926.000000,14926.000000,14926.000000,14926.000000,14926.000000,14926.000000,14926.000000,14926.000000,14926.000000,...,14926.000000,14926.000000,14926.000000,14926.000000,14926.000000,14926.000000,14926.000000,14926.000000,14926.000000,14926.000000
mean,0.115786,0.271538,0.092344,0.173864,0.105652,0.499777,0.368083,0.631917,0.000201,0.004489,...,0.072625,0.084215,0.070213,0.110813,0.070146,0.094466,0.070012,0.081804,0.620629,0.367743
std,0.034879,0.299315,0.241174,0.148803,0.110038,0.119862,0.482300,0.482300,0.014176,0.066850,...,0.259529,0.277720,0.255514,0.313911,0.255402,0.292486,0.255176,0.274074,0.098197,0.212455
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.096200,0.023694,0.000000,0.084196,0.039532,0.404943,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.568219,0.194664
50%,0.111811,0.139557,0.000021,0.113937,0.057492,0.495283,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.631436,0.345284
75%,0.130204,0.459839,0.005120,0.218542,0.120085,0.577509,1.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.682854,0.523929
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# 6) Feature reduction using VIF

In [ ]:
!pip install statsmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 13.2 MB/s eta 0:00:00


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

DROP=[];

for i in range(len(X_scaled.columns)):
    vif = pd.DataFrame()
    vif['Features'] = X_scaled.columns
    vif['VIF'] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    vif.reset_index(drop=True, inplace=True)

vif

,Features,VIF
0,time_signature_0,inf
1,time_signature_1,inf
2,audio_mode_0,inf
3,time_signature_3,inf
4,audio_mode_1,inf
5,key_3,inf
6,key_2,inf
7,key_1,inf
8,key_0,inf
9,time_signature_5,inf


### Observation from VIF table:
* features that has VIF more than 5 shall be removed.

In [ ]:
tobe_drop = vif[(vif['VIF'] > 5)]['Features'].tolist()
print(tobe_drop)

['time_signature_0', 'time_signature_1', 'audio_mode_0', 'time_signature_3', 'audio_mode_1', 'key_3', 'key_2', 'key_1', 'key_0', 'time_signature_5', 'time_signature_4', 'key_6', 'key_7', 'key_8', 'key_9', 'key_10', 'key_11', 'key_4', 'key_5']


In [ ]:
X_vif = X_scaled.drop(tobe_drop, axis=1)
print(X_vif.shape)
X_vif.head()

(14926, 8)


,song_duration_ms,acousticness,instrumentalness,liveness,speechiness,tempo,loudness_energy,danceability_av
0,0.140059,0.005541,0.000029,0.049226,0.031243,0.689425,0.714949,0.252207
1,0.114658,0.010340,0.000000,0.099580,0.052922,0.434371,0.539481,0.215128
2,0.122938,0.008202,0.448345,0.250333,0.084166,0.511233,0.660348,0.256159
3,0.114658,0.026505,0.003561,0.093426,0.113709,0.505303,0.583822,0.095794
4,0.118514,0.000957,0.000000,0.104707,0.033262,0.709856,0.643577,0.275243


# Split data into training and testing datasets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_vif, Y, train_size=0.8, test_size=0.2, random_state=40)
X_train.reset_index(drop=True,inplace=True)

print(">>> Size of trainig set: ", X_train.shape)
print(">>> Size of testing set: ", X_test.shape)

>>> Size of trainig set:  (11940, 8)
>>> Size of testing set:  (2986, 8)


# 6.  Train, fine tune, and evaluate model's performance

## Model Evaluation Functions

In [ ]:
# calculates r2_score
def cal_r2(y_true, y_pred):
  r2 = r2_score(y_true, y_pred)
  return r2

# Calculates adjusted_r2
def cal_adj_r2(x_df, r2):
  number_variables = x_df[1] - 1
  adjusted_r2 = 1 - ((1-r2) * (x_df[0]-1)) / (x_df[0] - number_variables -1)
  return adjusted_r2

def cal_mse(y_true, y_pred):
  mse = mean_squared_error(y_true, y_pred)
  return mse

def cal_performance(x_train_shape, x_test_shape, y_train, y_train_pred, y_test, y_test_pred):
  r2_train = cal_r2(y_train, y_train_pred)
  r2_test = cal_r2(y_test, y_test_pred)
  adj_r2_train = cal_adj_r2(x_train_shape, r2_train)
  adj_r2_test = cal_adj_r2(x_test_shape, r2_test)
  mse_train = cal_mse(y_train, y_train_pred)
  mse_test = cal_mse(y_test, y_test_pred)

  performance_dict = {
    'r2_train': r2_train,
    'r2_test': r2_test,
    'adj_r2_train': adj_r2_train,
    'adj_r2_test': adj_r2_test,
    'mse_train': mse_train,
    'mse_test': mse_test
    }
  return performance_dict

## 6a) Polynomial Regression (PR)

In [ ]:
# specify degree of 3 for polynomial regression model
# include bias=False means don't force y-intercept to equal zero
poly = PolynomialFeatures(degree=3, include_bias=False)
poly_train_features = poly.fit_transform(X_train)
poly_test_features = poly.fit_transform(X_test)

# Create LinearRegression
pr = LinearRegression()

# train the model
pr.fit(poly_train_features, y_train)

pr_pred_train = pr.predict(poly_train_features)
pr_pred_test = pr.predict(poly_test_features)

In [ ]:
pr_performance = cal_performance(X_train.shape, X_test.shape, y_train, pr_pred_train, y_test, pr_pred_test)
print(pr_performance)

{'r2_train': 0.06494537911562359, 'r2_test': 0.03131685110508187, 'adj_r2_train': 0.06439682209700215, 'adj_r2_test': 0.029039892729573302, 'mse_train': 388.25030973727866, 'mse_test': 402.58377880832694}


## 6b) Multiple Linear Regression (mlr)

In [ ]:
mlr = LinearRegression()
mlr_param = {'copy_X': [True, False], 'fit_intercept': [True, False], 'n_jobs': [1,5,10,15,None], 'positive': [True, False]}

random_search = RandomizedSearchCV(mlr, mlr_param, n_iter=100, cv=5)
random_search.fit(X_train, y_train)

# Parameter which gives the best results
print(f"Best Hyperparameters: {random_search.best_params_}")

# Accuracy of the model after using best parameters
print(f"Best Score: {random_search.best_score_}")

# Train the Elastic Net model with the best parameters
best_mlr = random_search.best_estimator_
best_mlr.fit(X_train, y_train)

# Predict on the test set
mlr_pred_train = best_mlr.predict(X_train)
mlr_pred_test = best_mlr.predict(X_test)

Best Hyperparameters: {'positive': False, 'n_jobs': 1, 'fit_intercept': True, 'copy_X': True}
Best Score: 0.024149656405073716


In [ ]:
# Evalute performance
mlr_performance = cal_performance(X_train.shape, X_test.shape, y_train, mlr_pred_train, y_test, mlr_pred_test)
print(mlr_performance)

{'r2_train': 0.026014456899828398, 'r2_test': 0.02273158197259595, 'adj_r2_train': 0.025443060754865243, 'adj_r2_test': 0.020434443313700146, 'mse_train': 404.41507944275816, 'mse_test': 406.15180834758917}


## 6c) Elastic Net Regression (enr)

In [ ]:
# Create an ElasticNet regression model instance
# l1_ratio corresponds to the mix of L1 and L2 regularization
# alpha corresponds to the strength of the regularization
enr = ElasticNet()

# Define the hyperparameters grid to search
param_grid = {
    'alpha': [0.1, 1, 10, 100],
    'l1_ratio': [0.1, 0.5, 0.7, 0.9, 1.0],
    'fit_intercept': [True, False],
    'max_iter': [100, 500, 1000]
}

# Perform GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(enr, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score (negative MSE): ", grid_search.best_score_)

# Train the Elastic Net model with the best parameters
best_elastic_net = grid_search.best_estimator_
best_elastic_net.fit(X_train, y_train)

# Predict on the test set
enr_pred_train = best_elastic_net.predict(X_train)
enr_pred_test = best_elastic_net.predict(X_test)

Best parameters found:  {'alpha': 0.1, 'fit_intercept': True, 'l1_ratio': 1.0, 'max_iter': 100}
Best cross-validation score (negative MSE):  -407.22216046834933


In [ ]:
# Evaluate the model
enr_performance = cal_performance(X_train.shape, X_test.shape, y_train, enr_pred_train, y_test, enr_pred_test)
print(enr_performance)

{'r2_train': 0.01993690070772436, 'r2_test': 0.020102229574576636, 'adj_r2_train': 0.01936193911745898, 'adj_r2_test': 0.017798910436572002, 'mse_train': 406.9385823711736, 'mse_test': 407.244564658485}


## 6d) Decision Tree Regressor (dtr)

In [ ]:
# Create a Decision Tree Regressor object
dtr = DecisionTreeRegressor(random_state=40)

# Fit the regressor to the training data
dtr.fit(X_train, y_train)

# Predict on the test data
dtr_pred_train = dtr.predict(X_train)
dtr_pred_test = dtr.predict(X_test)



In [ ]:
# Evaluate the model
dtr_performance = cal_performance(X_train.shape, X_test.shape, y_train, dtr_pred_train, y_test, dtr_pred_test)
print(dtr_performance)

{'r2_train': 0.985016468830553, 'r2_test': -0.9172750368656923, 'adj_r2_train': 0.985007678626213, 'adj_r2_test': -0.9217817276843827, 'mse_train': 6.221412618648799, 'mse_test': 796.817649028801}


## 6e) Adaboost (ada)

In [ ]:
# Define the AdaBoost Regressor model
adaboost = AdaBoostRegressor() # base estimator is DecisionTreeRegressor

# Define the hyperparameters grid to search
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.05, 1, 0.2, 0.5, 0.7, 1.0],
    'loss': ['linear', 'square', 'exponential']
}

# Perform GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(adaboost, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score (negative MSE): ", grid_search.best_score_)

# Train the AdaBoost Regressor model with the best parameters
best_adaboost = grid_search.best_estimator_
best_adaboost.fit(X_train, y_train)

# Predict on the test data
ada_pred_train = best_adaboost.predict(X_train)
ada_pred_test = best_adaboost.predict(X_test)

Best parameters found:  {'learning_rate': 0.01, 'loss': 'square', 'n_estimators': 50}
Best cross-validation score (negative MSE):  -400.94075016764145


In [ ]:
  # Evaluate the model
  ada_performance = cal_performance(X_train.shape, X_test.shape, y_train, ada_pred_train, y_test, ada_pred_test)
  print(ada_performance)

{'r2_train': 0.04287149879157415, 'r2_test': 0.03787242360953491, 'adj_r2_train': 0.04230999196049312, 'adj_r2_test': 0.03561087457167944, 'mse_train': 397.41575385305674, 'mse_test': 399.85928922253675}


## 6f) Gradient Boosting Regressor (gbr)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Create a Gradient Boosting Regressor model
gbr = GradientBoostingRegressor()

# Define the hyperparameters grid to search
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.05, 1, 0.2, 0.5, 0.7, 1.0],
    'max_depth': [3,5,7,9,11,13,15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Perform GridSearchCV to find the best hyperparameters
random_search = RandomizedSearchCV(gbr, param_distributions=param_grid, n_iter=100, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)


# Print the best parameters and the best score
print("Best parameters found: ", random_search.best_params_)
print("Best cross-validation score (negative MSE): ", random_search.best_score_)

# Train the AdaBoost Regressor model with the best parameters
best_gbr = random_search.best_estimator_
best_gbr.fit(X_train, y_train)

# Predict on the test data
gbr_pred_train = best_gbr.predict(X_train)
gbr_pred_test = best_gbr.predict(X_test)


Best parameters found:  {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_depth': 3, 'learning_rate': 0.05}
Best cross-validation score (negative MSE):  -395.6940886717014


In [ ]:
# Evaluate the model
gbr_performance = cal_performance(X_train.shape, X_test.shape, y_train, gbr_pred_train, y_test, gbr_pred_test)
print(gbr_performance)

{'r2_train': 0.08164975648164619, 'r2_test': 0.0473010043707649, 'adj_r2_train': 0.08111099921508325, 'adj_r2_test': 0.04506161788003127, 'mse_train': 381.31437301072395, 'mse_test': 395.94078018685695}


## 6g) RBF SVC (RBF)

In [ ]:
# Define the SVR model with RBF kernel
svr = SVR(kernel='rbf')

# Define the hyperparameters grid to search
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1e-3, 1e-2, 1e-1, 1]
}

# Perform GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(svr, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score (negative MSE): ", grid_search.best_score_)

# Train the SVR model with the best parameters
best_svr = grid_search.best_estimator_
best_svr.fit(X_train, y_train)

# Predict on the test set
rbf_pred_train = best_svr.predict(X_train)
rbf_pred_test = best_svr.predict(X_test)

Best parameters found:  {'C': 10, 'gamma': 1}
Best cross-validation score (negative MSE):  -406.55344071890676


In [ ]:
# Evaluate the model
rbf_performance = cal_performance(X_train.shape, X_test.shape, y_train, rbf_pred_train, y_test, rbf_pred_test)
print(rbf_performance)

{'r2_train': 0.029688175691088503, 'r2_test': 0.019386167192908532, 'adj_r2_train': 0.029118934761641402, 'adj_r2_test': 0.01708116489954059, 'mse_train': 402.88968998770645, 'mse_test': 407.54215949101956}


## 6h) Perform stacking on base models (STA)

In [ ]:
# Define the base estimators
#base_estimators = [pr, best_mlr, best_elastic_net, best_adaboost, best_gbr, best_svr]

base_estimators = [
    ('pr', pr),
    ('enr', best_elastic_net),
    ('ada', best_adaboost),
    ('gbr', best_gbr),
    ('rbf',best_svr)
]

# Create the Stacking Regressor with a Linear Regression meta-regressor
stacking_regressor = StackingRegressor(
    estimators=base_estimators,
    cv=5
)

# Fit the stacking regressor
stacking_regressor.fit(X_train, y_train)

# Predict on the test data
stack_pred_train = stacking_regressor.predict(X_train)
stack_pred_test = stacking_regressor.predict(X_test)

In [ ]:
# Evaluate the model
stacking_performance = cal_performance(X_train.shape, X_test.shape, y_train, stack_pred_train, y_test, stack_pred_test)
print(stacking_performance)

{'r2_train': 0.0805399131638449, 'r2_test': 0.05176132757647667, 'adj_r2_train': 0.08000050479912368, 'adj_r2_test': 0.049532425391465074, 'mse_train': 381.77519850933345, 'mse_test': 394.08707418101756}


## Compare performance

In [ ]:
# convert dictionary to dataframe
pr_df  = pd.DataFrame.from_dict(pr_performance, orient='index', columns=['PR'])
mlr_df = pd.DataFrame.from_dict(mlr_performance, orient='index', columns=['MLR'])
enr_df = pd.DataFrame.from_dict(enr_performance, orient='index', columns=['ENR'])
dtr_df = pd.DataFrame.from_dict(dtr_performance, orient='index', columns=['DTR'])
ada_df = pd.DataFrame.from_dict(ada_performance, orient='index', columns=['ADA'])
gbr_df = pd.DataFrame.from_dict(gbr_performance, orient='index', columns=['GBR'])
rbf_df = pd.DataFrame.from_dict(rbf_performance, orient='index', columns=['RBF'])
stack_df = pd.DataFrame.from_dict(stacking_performance, orient='index', columns=['STA'])

df_performance = pd.concat([pr_df, mlr_df, enr_df, dtr_df, ada_df, gbr_df, rbf_df, stack_df], axis=1)
print(">>> Comparision of model performance:\n")
df_performance

>>> Comparision of model performance:



,PR,MLR,ENR,DTR,ADA,GBR,RBF,STA
r2_train,0.064945,0.026014,0.019937,0.985016,0.042871,0.081650,0.029688,0.080540
r2_test,0.031317,0.022732,0.020102,-0.917275,0.037872,0.047301,0.019386,0.051761
adj_r2_train,0.064397,0.025443,0.019362,0.985008,0.042310,0.081111,0.029119,0.080001
adj_r2_test,0.029040,0.020434,0.017799,-0.921782,0.035611,0.045062,0.017081,0.049532
mse_train,388.250310,404.415079,406.938582,6.221413,397.415754,381.314373,402.889690,381.775199
mse_test,402.583779,406.151808,407.244565,796.817649,399.859289,395.940780,407.542159,394.087074


In [ ]:
print(y_test[0:20].values)
print(stack_pred_test[:20])

[55 51 72 56 73 58 60 16 38 95  4 51 67 55 60 69 56 28 49 61]
[50.21395522 52.01357306 45.43179347 49.71750579 51.41299971 49.3812037
 49.27276966 45.04644579 46.96830539 55.87578934 51.64202918 39.6247286
 50.7202061  46.20268663 45.73552324 41.39816296 47.91347784 52.86128038
 49.08225369 52.59854835]


# 7.0 Deployment

## 7.1 Save Your Model with Pickle

In [ ]:
import pickle

# Save the trained stacking model
with open('stacking_model.pkl', 'wb') as f:
    pickle.dump(stacking_regressor, f)


## 7.2 Set Up a Gradio Interface

In [ ]:
!pip install --upgrade gradio


In [ ]:
import gradio as gr
import pickle
import numpy as np

# Load the trained model
with open('stacking_model.pkl', 'rb') as f:
    model = pickle.load(f)

# Define the feature names based on the final set of features
feature_names = ['song_duration_ms', 'acousticness', 'instrumentalness', 'liveness', 'speechiness', 'tempo', 'loudness_energy', 'danceability_av']

# Define the Gradio interface
def predict(*features):
    # Convert input features to numpy array and reshape for prediction
    features = np.array(features).reshape(1, -1)
    prediction = model.predict(features)
    return prediction[0]

# Define sliders with appropriate min, max, and default values
inputs = [gr.Slider(minimum=0, maximum=1, value=0.5, label=feature) for feature in feature_names]
outputs = gr.Textbox(label="Predicted Popularity")

# Create the Gradio interface
interface = gr.Interface(fn=predict, inputs=inputs, outputs=outputs)

# Launch the interface
interface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f56d3147225ac4cc4e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
